# Proyecto de Telecomunicaciones: Identificar operadores ineficaces

El servicio de telefonía virtual CallMeMaybe está desarrollando una nueva función que brindará a los supervisores y las supervisores información sobre los operadores menos eficaces. Se considera que un operador es ineficaz si tiene una gran cantidad de llamadas entrantes perdidas (internas y externas) y un tiempo de espera prolongado para las llamadas entrantes. Además, si se supone que un operador debe realizar llamadas salientes, un número reducido de ellas también será un signo de ineficacia.

# Fases del proyecto

- Objetivo de proyecto: Identificar a los operadores más ineficaces para poder realizar acciones especificas en ese personal y poder mejorar el rendimiento del servicio.

1. Descarga de datos y estudio de la información general
- En esta fase se descargaran los datos a ocupar y se vera una muestra de ellos, junto con información general (tipos de datos, columnas, etc) para identificar ajustes a realizar en la siguiente fase
2. Preprocesamiento de datos
- En esta fase se realizaran los ajustes previamente identificados y se prepara los datos para el análisis.
3. Análisis Exploratorio de los datos (EDA)
- Describe el comportamiento de los operadores. Identifica cuantos operadores están activos, su número de llamadas tanto salientes como entrantes, número total de llamadas y llamadas perdidas, calcula el tiempo de espera y tiempo de duración de llamadas promedio.
- Describe el comportamiento de los clientes. Identifica que clientes ocupan más el servicio, su tarifa actual, que tanto tiempo llevan con el servicio.
4. Análisis
- Analiza e identifica a los 5 operadores que menos eficacia tienen respecto al resto.
- Analiza e identifica a los 5 operadores que más eficacia tienen respecto al resto
5. Prueba las hipótesis
- Prueba que el número promedio de llamadas entrantes perdidas de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
- Prueba que el tiempo de espera promedio de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
- Prueba que el número promedio de llamadas salientes de los operadores identificados como menos eficaces y de los operadores identificados como más eficaces difiere.
6. Escribe conclusiones generales y tus recomendaciones finales y posibles siguientes pasos.
- Comenta las conclusiones y recomendaciones finales de todo el proceso.
- Escribe los posibles next steps basados en tus conclusiones y recomendaciones finales

## Tabla de contenidos <a id='back'></a>

- [Descarga de datos y estudio de la información general](#descarga_datos)
- [Preprocesamiento de datos](#preprocesamiento)
- [Análisis exploratorio de datos (EDA)](#eda)
- [Prueba las hipótesis](#hipotesis)
- [Conclusiones y recomendaciones](#conclusiones_recomendaciones)

## Descarga de datos y estudio de la información general

En esta fase se descargaran los datos a ocupar y se vera una muestra de ellos, junto con información general (tipos de datos, columnas, etc) para identificar ajustes a realizar en la siguiente fase


In [2]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from plotly import graph_objects as go
from scipy import stats as st

Vamos a cargar los datos de los siguientes datasets:

- `telecom_clients_us.csv`
- `telecom_dataset_us.csv`

In [6]:
# Carga de los archivos de datos
data_clients = pd.read_csv('data/telecom_clients_us.csv')
data = pd.read_csv('data/telecom_dataset_us.csv')

# Verificamos que los datos se hayan cargado correctamente
print(data_clients.head())
print('\n')
print(data.head())

   user_id tariff_plan  date_start
0   166713           A  2019-08-15
1   166901           A  2019-08-23
2   168527           A  2019-10-29
3   167097           A  2019-09-01
4   168193           A  2019-10-16


   user_id                       date direction internal  operator_id  \
0   166377  2019-08-04 00:00:00+03:00        in    False          NaN   
1   166377  2019-08-05 00:00:00+03:00       out     True     880022.0   
2   166377  2019-08-05 00:00:00+03:00       out     True     880020.0   
3   166377  2019-08-05 00:00:00+03:00       out     True     880020.0   
4   166377  2019-08-05 00:00:00+03:00       out    False     880022.0   

   is_missed_call  calls_count  call_duration  total_call_duration  
0            True            2              0                    4  
1            True            3              0                    5  
2            True            1              0                    1  
3           False            1             10                   18  
4   

Los datos se han cargado correctamente. Vamos a estudiar los datos que continen nuestros datasets y verificar que los tipos de datos sean correctos para nuestro análisis

In [7]:
# Imprimimos la información general/resumen sobre nuestro primer dataset data_clients
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      732 non-null    int64 
 1   tariff_plan  732 non-null    object
 2   date_start   732 non-null    object
dtypes: int64(1), object(2)
memory usage: 17.3+ KB


In [9]:
data_clients.head()

,user_id,tariff_plan,date_start
0,166713,A,2019-08-15
1,166901,A,2019-08-23
2,168527,A,2019-10-29
3,167097,A,2019-09-01
4,168193,A,2019-10-16


Nuestro dataset `data_clients` que son los datos de los clientes del servicio, contiene los siguientes datos:

- `user_id`: ID de usuario/a
- `tariff_plan`: tarifa actual de la clientela
- `date_start`: fecha de registro de la clientela

In [12]:
# Verificamos que no haya datos duplicados en el dataset completo
print('Datos duplicados en el dataset:', data_clients.duplicated().sum())
# Verificamos que no haya datos duplicados en user_id
print('Datos duplicados en el user_id del dataset:', data_clients['user_id'].duplicated().sum())

Datos duplicados en el dataset: 0
Datos duplicados en el user_id del dataset: 0


### Conclusiones del dataset

Al ver la muestra y resumen del dataset, no encontramos duplicados ni ausentes. Sin embargo, es necesario cambiar el tipo de dato de la columna `date_start` a datetime para poder realizar operaciones con fechas.

In [13]:
# Imprimimos la información general/resumen sobre nuestro segundo dataset data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53902 entries, 0 to 53901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              53902 non-null  int64  
 1   date                 53902 non-null  object 
 2   direction            53902 non-null  object 
 3   internal             53785 non-null  object 
 4   operator_id          45730 non-null  float64
 5   is_missed_call       53902 non-null  bool   
 6   calls_count          53902 non-null  int64  
 7   call_duration        53902 non-null  int64  
 8   total_call_duration  53902 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.3+ MB


In [14]:
data.head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
1,166377,2019-08-05 00:00:00+03:00,out,True,880022.0,True,3,0,5
2,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,True,1,0,1
3,166377,2019-08-05 00:00:00+03:00,out,True,880020.0,False,1,10,18
4,166377,2019-08-05 00:00:00+03:00,out,False,880022.0,True,3,0,25


Nuestro dataset `data` que son los datos sobre las llamadas realizadas por los operadores o recibidas por ellos, contiene las siguientes columnas:

- `user_id`: ID de la cuenta de cliente
- `date`: fecha en la que se recuperaron las estadísticas
- `direction`: - "dirección" de llamada (`out` para saliente, `in` para entrante)
- `internal`: si la llamada fue interna (entre los operadores de un cliente o clienta)
- `operator_id`: identificador del operador
- `is_missed_call`: si fue una llamada perdida
- `calls_count`: número de llamadas
- `call_duration`: duración de la llamada (sin incluir el tiempo de espera)
- `total_call_duration`: duración de la llamada (incluido el tiempo de espera)

In [17]:
# Verificamos que no haya datos duplicados en el dataset completo
print('Datos duplicados en el dataset:', data.duplicated().sum())

Datos duplicados en el dataset: 4900


### Conclusiones del dataset

Al ver la muestra y resumen del dataset, encontramos 2 columnas con datos ausentes `internal` y `operator_id`; por la muestra que vimos de los datos si la llamada es dirección entrante el operator_id no es registrado y de ahi los nulos de esta columna, sin embargo, se debe hacer un análisis a fondo para tomar la mejor decision para tratarlos. Junto a esto, se detectaron 4900 datos duplicados, estos también deben investigarse para tomar la mejor decisión de como tratarlos. Finalmente se deben cambiar los tipos de datos de las siguientes columnas `date` a datetime, `internal` a bool y `operator_id` a int.

## Preprocesamiento de datos

En esta fase haremos los ajustes identificados en la fase anterior, los cuales se encuentran ambos datasets.

### Datos Ausentes

Como comentamos anteriormente, se encontraron 2 columnas con datos ausentes

- `internal`
- `operator_id`

Investiguemos las columnas para tomar la mejor decisión de como procesarlos.

In [18]:
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.user_id.size
data.isna().sum().apply(porcentaje_nulos)

user_id                 0.000000
date                    0.000000
direction               0.000000
internal                0.217061
operator_id            15.160847
is_missed_call          0.000000
calls_count             0.000000
call_duration           0.000000
total_call_duration     0.000000
dtype: float64

In [20]:
## Investiguemos la columna internal
# Veamos una muestra de los datos donde hay ausentes
data[data['internal'].isna()].head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
1007,166405,2019-09-18 00:00:00+03:00,in,NaN,NaN,True,1,0,59
1090,166405,2019-10-01 00:00:00+03:00,in,NaN,NaN,True,1,0,1
1864,166406,2019-08-20 00:00:00+03:00,in,NaN,NaN,True,1,0,36
1924,166406,2019-09-02 00:00:00+03:00,in,NaN,879898.0,False,1,2,9
6210,166541,2019-09-26 00:00:00+03:00,in,NaN,908960.0,False,1,393,423


Parece indicar que todas las llamadas internas vacías son llamadas entrantes, verifiquemos.

In [21]:
# Vemos los valores en direction de las rows con valores ausentes en internal
data[data['internal'].isna()]['direction'].value_counts()

in     115
out      2
Name: direction, dtype: int64

Como podemos observar, tenemos 115 llamadas entrantes y 2 salientes; por lo que nuestra hipotesis del inicio no fue cierta. Tomando en cuenta que sin otra posible conexion con las demás columnas no seria posible recuperar los posibles datos faltantes y que los datos ausentes representan un 0.21% de nuestra total data, se toma la decisión de eliminar los datos ausentes de esta columna.

In [37]:
# Eliminamos los datos ausentes de la columna internal
data = data[~data['internal'].isna()]
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.user_id.size
data.isna().sum().apply(porcentaje_nulos)

user_id                 0.00000
date                    0.00000
direction               0.00000
internal                0.00000
operator_id            15.08785
is_missed_call          0.00000
calls_count             0.00000
call_duration           0.00000
total_call_duration     0.00000
dtype: float64

In [39]:
## Investiguemos la columna operator_id
# Veamos una muestra de los datos donde hay ausentes
data[data['operator_id'].isna()].head()

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
0,166377,2019-08-04 00:00:00+03:00,in,False,NaN,True,2,0,4
7,166377,2019-08-05 00:00:00+03:00,in,False,NaN,True,6,0,35
9,166377,2019-08-06 00:00:00+03:00,in,False,NaN,True,4,0,62
17,166377,2019-08-07 00:00:00+03:00,in,False,NaN,True,2,0,24
27,166377,2019-08-12 00:00:00+03:00,in,False,NaN,True,2,0,34


In [43]:
# Veamos los valores unicos de las columnas direction, internal, is_missed_call y user_id
print('Direction')
print(data[data['operator_id'].isna()]['direction'].value_counts())
print('\nInternal')
print(data[data['operator_id'].isna()]['internal'].value_counts())
print('\nIs_missed_call')
print(data[data['operator_id'].isna()]['is_missed_call'].value_counts())
print('\nuser_id')
print(data[data['operator_id'].isna()]['user_id'].value_counts())

Direction
in     7917
out     198
Name: direction, dtype: int64

Internal
False    7760
True      355
Name: internal, dtype: int64

Is_missed_call
True     7993
False     122
Name: is_missed_call, dtype: int64

user_id
168252    145
166405    120
166582    118
166782    117
166658    111
         ... 
168010      1
167139      1
166507      1
166554      1
168013      1
Name: user_id, Length: 305, dtype: int64


Como podemos ver, las primeras 3 columnas tienen datos ausentes en las 2 opciones por lo que no podriamos generar una relación desde ellas para rellenar estos datos, sin embargo podemos ver que tenemos 305 clientes sin operador_id y si cada cliente tiene un unico operador, podemos generar una relación y recuperar los datos. Primero verifiquemos que cada cliente tiene un solo operador.

In [44]:
# Verificamos si cada cliente tiene un solo operador
data.groupby('user_id')['operator_id'].nunique()

user_id
166377     5
166391     2
166392     3
166399     1
166405    10
          ..
168583     2
168598     1
168601     2
168603     1
168606     1
Name: operator_id, Length: 307, dtype: int64

Como podemos ver en la tabla, hay clientes que tiene hasta 10 operadores por lo que no es posible saber con seguridad que operador cubrio la llamada. Tomando en cuenta todo esto se decide por ultimo eliminarlos del dataset, la perdida seria del 15% de nuestra información, sin embargo, al no poder recuperar los datos del operador no podriamos realizar un análisis correctamente.

In [45]:
# Eliminamos los datos ausentes de la columna internal
data = data[~data['operator_id'].isna()]
# Vamos a checar el porcentaje de valores ausentes en general
porcentaje_nulos = lambda x: x * 100 / data.user_id.size
data.isna().sum().apply(porcentaje_nulos)

user_id                0.0
date                   0.0
direction              0.0
internal               0.0
operator_id            0.0
is_missed_call         0.0
calls_count            0.0
call_duration          0.0
total_call_duration    0.0
dtype: float64

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45670 entries, 1 to 53900
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              45670 non-null  int64  
 1   date                 45670 non-null  object 
 2   direction            45670 non-null  object 
 3   internal             45670 non-null  object 
 4   operator_id          45670 non-null  float64
 5   is_missed_call       45670 non-null  bool   
 6   calls_count          45670 non-null  int64  
 7   call_duration        45670 non-null  int64  
 8   total_call_duration  45670 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(3)
memory usage: 3.2+ MB


Al eliminar los datos ausentes, pasamos de 53,902 rows a 45,670 rows, un número todavia apto para realizar nuestro análisis.

### Datos duplicados

Se detectaron un total de 4,900 datos duplicados, los cuales deben estudiarse para ver la mejor opción para tratarlos

In [50]:
# Veamos una muestra de los datos donde hay ausentes
data[data.duplicated()]

,user_id,date,direction,internal,operator_id,is_missed_call,calls_count,call_duration,total_call_duration
8,166377,2019-08-05 00:00:00+03:00,out,False,880020.0,True,8,0,50
44,166377,2019-08-14 00:00:00+03:00,out,False,880026.0,False,10,1567,1654
51,166377,2019-08-15 00:00:00+03:00,out,False,880026.0,False,11,1413,1473
62,166377,2019-08-19 00:00:00+03:00,out,False,880026.0,False,14,1519,1598
78,166377,2019-08-22 00:00:00+03:00,out,False,880026.0,True,6,0,55
...,...,...,...,...,...,...,...,...,...
53861,168601,2019-11-20 00:00:00+03:00,out,False,952914.0,True,1,0,2
53869,168601,2019-11-25 00:00:00+03:00,in,False,952914.0,False,7,1229,1282
53874,168601,2019-11-26 00:00:00+03:00,in,False,952914.0,False,4,539,562
53885,168603,2019-11-20 00:00:00+03:00,out,False,959118.0,True,3,0,89


Una vez viendo una muestra podemos ver que los datos que se encontraron duplicados fue en su mayoria porque compartian user_id, operator_id y otras columnas que tienden a repetirse debido a la naturaleza de como estan los datos, sin embargo las fechas difieren para cada columna.